# Imports

In [ ]:
!pip install faiss-cpu
!pip install faiss-gpu-cu12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 99.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import pipeline
from typing import List

# RAG Dataset

## JSON QA Pairs

In [ ]:
with open("qa_pairs.json", "r", encoding="utf-8") as f:
    qa_data = json.load(f)["questions"]

In [ ]:
qa_data[:5]

[{'question': 'What are the profit payment and profit rate for NUST Asaan Account Savings Account?',
  'answer': 'Profit Payment is Semi-Annually, Profit Rate is 19.00%.'},
 {'question': 'What are the profit payment and profit rate for Profit Loss Sharing Savings Account?',
  'answer': 'Profit Payment is Semi-Annually, Profit Rate is 19.00%.'},
 {'question': 'What are the profit payment and profit rate for Little Champs Account Savings Account?',
  'answer': 'Profit Payment is Semi-Annually, Profit Rate is 19.00%.'},
 {'question': 'What are the profit payment and profit rate for NUST Special Deposit Account (ASDA) Savings Account?',
  'answer': 'Profit Payment is Monthly/Semi-Annually, Profit Rate is 19.00%.'},
 {'question': 'What are the profit payment and profit rate for NUST Waqaar Account - Senior Citizen Savings Account?',
  'answer': 'Profit Payment is Monthly, Profit Rate is 19.00%.'}]

## Chunking

In [ ]:
documents = [f"Q: {item['question']}\nA: {item['answer']}" for item in qa_data]

In [ ]:
documents[:5]

['Q: What are the profit payment and profit rate for NUST Asaan Account Savings Account?\nA: Profit Payment is Semi-Annually, Profit Rate is 19.00%.',
 'Q: What are the profit payment and profit rate for Profit Loss Sharing Savings Account?\nA: Profit Payment is Semi-Annually, Profit Rate is 19.00%.',
 'Q: What are the profit payment and profit rate for Little Champs Account Savings Account?\nA: Profit Payment is Semi-Annually, Profit Rate is 19.00%.',
 'Q: What are the profit payment and profit rate for NUST Special Deposit Account (ASDA) Savings Account?\nA: Profit Payment is Monthly/Semi-Annually, Profit Rate is 19.00%.',
 'Q: What are the profit payment and profit rate for NUST Waqaar Account - Senior Citizen Savings Account?\nA: Profit Payment is Monthly, Profit Rate is 19.00%.']

In [ ]:
len(documents)

327

# Embedding Model

In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings = embedding_model.encode(documents, convert_to_numpy=True)

In [ ]:
embeddings[0][:10]

array([ 0.02455929,  0.08280013, -0.05625945, -0.04772451,  0.02212636,
        0.04610027,  0.01438646,  0.04925155,  0.0171691 ,  0.03099133],
      dtype=float32)

# FAISS Index

## Building

In [ ]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

## Persisting

In [ ]:
faiss.write_index(index, "bank-data_index.faiss")

In [ ]:
index = faiss.read_index("bank-data_index.faiss")

# Retrieval

## Retireving Docs

In [ ]:
def retrieve(query: str, top_k=3) -> List[str]:
    query_vec = embedding_model.encode([query])
    distances, indices = index.search(query_vec, top_k)
    return [documents[i] for i in indices[0]]

## Instruct Model

In [ ]:
model_name = "Qwen/Qwen3-4B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

tokenizer_config.json:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

# Querying

## Prompting

In [ ]:
def build_prompt(query: str, context_docs: List[str]) -> str:
    context = "\n---\n".join(context_docs)
    return (
        f"<|system|>>\nYou are a helpful banking assistant. You are provided with NUST Bank FAQ relevant to the query of the user. Your job is to give direct answers according to the provided context. NOT from your own knowledge. Just give the answer in a user friendly manner nothing more\n"
        f"Context:\n\n{context}\n"
        f"---\n</s>"
        f"<|user|>\n {query}\n</s>"
        f"<|assistant|>"
    )

## Asking

In [ ]:
def ask(query: str):
    context_docs = retrieve(query)
    prompt = build_prompt(query, context_docs)
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.to(model.device)
    output_ids = model.generate(input_ids, max_new_tokens=200, temperature=0.7)
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(f"Answer:\n{answer}")

In [ ]:
ask("What is the profit rate for NUST Maximiser Savings Account?")

Answer:
<|system|>>
You are a helpful banking assistant. You are provided with NUST Bank FAQ relevant to the query of the user. Your job is to give direct answers according to the provided context. NOT from your own knowledge. Just give the answer in a user friendly manner nothing more
Context:

Q: What are the profit payment and profit rate for NUST Maximiser Savings Account?
A: When Profit Payment is Monthly, Profit Rate is 19.00%. When Profit Payment is Quarterly, Profit Rate is 19.05%. When Profit Payment is Semi-Annually, Profit Rate is 19.10%. When Profit Payment is Annually, Profit Rate is 19.15%.
---
Q: What are the profit payment and profit rate for NUST Asaan Account Savings Account?
A: Profit Payment is Semi-Annually, Profit Rate is 19.00%.
---
Q: What are the main Features of NUST Maximiser Savings Account? for NUST Maximiser Account
A: -  Attractive profit payment on monthly, quarterly, semi-annual & annual basis: - Minimum monthly average balance requirement Rs. 100,000 -